In [ ]:
from inspect_ai.log import (
    list_eval_logs,
    read_eval_log,
    read_eval_log_samples,
    read_eval_log_sample,
)

logs = list_eval_logs()

log = read_eval_log(logs[1], header_only=True)
log

EvalLog(version=2, status='cancelled', eval=EvalSpec(run_id='SCscU7iUgXxJMitxqEksnt', created='2024-12-11T00:00:18+00:00', task='inspect_evals/pubmedqa', task_id='fr3oEMNZf7ZSRYpS5eCrak', task_version=0, task_file=None, task_attribs={}, task_args={}, solver=None, solver_args=None, tags=None, dataset=EvalDataset(name='bigbio/pubmed_qa', location='bigbio/pubmed_qa', samples=500, sample_ids=['12377809', '26163474', '19100463', '18537964', '12913878', '12765819', '25475395', '19130332', '9427037', '24481006', '8165771', '22680064', '22540518', '20629769', '21726930', '21481154', '22902073', '26370095', '18041059', '15041506', '11146778', '27281318', '21645374', '9465206', '25887165', '15995461', '21850494', '19106867', '21342862', '24352924', '16147837', '26879871', '15918864', '22075911', '11035130', '21228436', '11833948', '17682349', '17355582', '15597845', '10158597', '27549226', '26348845', '25588461', '23359100', '26548832', '25756710', '20297950', '24622801', '9722752', '20577124', 

In [ ]:
for sample in read_eval_log_samples(logs[0], all_samples_required=False):
    print(sample)
    print(f"Sample id: {sample['id']}")
    print(f"Sample
    break

id='0' epoch=1 input=[ChatMessageUser(content='You are a helpful AI assistant respond to the following questions based on the context provided.\n\n<P> The fourth season of Chicago Fire , an American drama television series with executive producer Dick Wolf , and producers Derek Haas , Michael Brandt , and Matt Olmstead , was ordered on February 5 , 2015 , by NBC , and premiered on October 13 , 2015 and concluded on May 17 , 2016 . The season contained 23 episodes . </P>\n\nhow many episodes are in chicago fire season 4', source=None, role='user')] choices=None target="['23']" sandbox=None files=None setup=None messages=[ChatMessageUser(content='You are a helpful AI assistant respond to the following questions based on the context provided.\n\n<P> The fourth season of Chicago Fire , an American drama television series with executive producer Dick Wolf , and producers Derek Haas , Michael Brandt , and Matt Olmstead , was ordered on February 5 , 2015 , by NBC , and premiered on October 13